# LangExtract 信息抽取笔记本

本笔记本演示如何使用 `langextract` 对本仓库 `data/` 中示例文档进行结构化信息抽取（以合同/仲裁文本为例）。

提示：首次运行前，请确保安装依赖并配置可用的大模型 API Key。

In [ ]:
# 安装依赖（首次运行取消注释后执行）
# %pip install -U langextract pypdf pandas
# 如果你使用代理或自建网关，请在下方配置 BASE_URL。

In [ ]:
import os, json, sys
from pathlib import Path

try:
    import langextract
    print("langextract version:", getattr(langextract, "__version__", "unknown"))
except ModuleNotFoundError:
    print("未安装 langextract。请先运行上一个单元安装依赖。")

print("Python:", sys.version)

In [ ]:
# 基础配置：优先从环境变量读取
BASE_URL = os.environ.get('OPENAI_BASE_URL') or os.environ.get('BASE_URL') or ''
API_KEY = os.environ.get('OPENAI_API_KEY') or os.environ.get('API_KEY') or ''
MODEL = os.environ.get('OPENAI_MODEL') or 'gpt-4o-mini'

print('Using model:', MODEL)
if not API_KEY:
    print('警告：未检测到 API Key，请设置 OPENAI_API_KEY 或在此处直接填写 API_KEY 变量。')
if BASE_URL:
    print('Using base url:', BASE_URL)

In [ ]:
# 定义抽取 Schema（JSON Schema 形式，通用且直观）
schema = {
    'type': 'object',
    'title': 'ContractInfo',
    'properties': {
        'title': { 'type': 'string', 'description': '文档标题或合同名称' },
        'parties': { 'type': 'array', 'items': { 'type': 'string' }, 'description': '合同双方或多方名称' },
        'effective_date': { 'type': 'string', 'description': '生效日期（原文格式）' },
        'amount': { 'type': 'string', 'description': '金额（含币种），原文格式' },
        'duration': { 'type': 'string', 'description': '合同期限' },
        'key_clauses': { 'type': 'array', 'items': { 'type': 'string' }, 'description': '关键条款摘要' }
    },
    'required': ['title', 'parties'],
    'additionalProperties': False
}

system_prompt = (
    '你是信息抽取助手。根据给定 Schema 从输入文本中抽取结构化字段。'
    '严格输出 JSON，且仅包含指定键；无法确定时可输出空字符串或空数组。'
)

In [ ]:
# 读取示例文档（优先选择合同示例，其次仲裁/论文示例）
DATA_DIR = Path('data')
candidates = [
    DATA_DIR / '示例5-合同范本.md',
    DATA_DIR / '示例2-仲裁文件.md',
    DATA_DIR / '示例1-期刊论文.md',
]
example = next((p for p in candidates if p.exists()), None)
assert example is not None, '未找到示例文件，请确认 data/ 目录下存在示例 Markdown 文档。'

text = example.read_text(encoding='utf-8', errors='ignore')
print('Loaded:', example, 'chars:', len(text))

In [ ]:
# 使用 langextract 进行抽取（不同版本 API 可能差异，以下提供两种常见写法）
result = None
try:
    from langextract import Extractor
    extractor = Extractor(model=MODEL, api_key=API_KEY, base_url=BASE_URL or None)
    # 假设 extract 接口支持 schema 与 system prompt
    result = extractor.extract(text, schema=schema, system_prompt=system_prompt)
except Exception as e:
    print('Extractor.extract 方式失败：', repr(e))
    try:
        from langextract import create_extractor
        extractor = create_extractor(model=MODEL, api_key=API_KEY, base_url=BASE_URL or None)
        result = extractor(text=text, schema=schema, prompt=system_prompt)
    except Exception as e2:
        print('create_extractor 方式也失败：', repr(e2))

if result:
    print(json.dumps(result, ensure_ascii=False, indent=2))
else:
    print('未得到抽取结果，请根据你使用的 langextract 版本调整上述调用。')

In [ ]:
# 保存抽取结果
OUT_DIR = Path('信息抽取') / 'output'
OUT_DIR.mkdir(parents=True, exist_ok=True)
out_path = OUT_DIR / f'langextract_{example.stem}.json'

if 'result' in globals() and result:
    out_path.write_text(json.dumps(result, ensure_ascii=False, indent=2), encoding='utf-8')
    print('已保存：', out_path.resolve())
else:
    print('无结果可保存。')

## 注意事项
- 不同版本的 `langextract` API 可能有所差异；若上面的导入或调用失败，请查看你本地安装版本的文档并对照修改。
- 默认使用 `OPENAI_API_KEY` 和可选的 `OPENAI_BASE_URL`/`BASE_URL`；如需使用其他服务商，请在对应参数中切换。
- 若需要处理 PDF，可安装 `pypdf` 并自行添加解析逻辑（本示例优先使用已有的 Markdown 文件）。
- 结果将保存至 `信息抽取/output/` 目录，便于后续评估与比对。